In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
import fastai
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

import torch.nn as nn 
import torch 
import numpy 
%matplotlib inline

from torch.autograd import Variable
from torchvision import models
from fastai import conv_learner
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform

In [4]:
# Parameters for respective architectures
arch1 = resnet50
PATH1 = "../../CHRC_NEW_DATA/chrc_data_patches_norm_1000_dup/"
sz1=256


arch2=resnet50
PATH2 = "../../CHRC_NEW_DATA/chrc_data_patches_norm_duplicated/"
sz2=512

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.backends.cudnn.enabled

True

In [7]:
data1 = ImageClassifierData.from_paths(PATH1, tfms=tfms_from_model(arch1, sz1))
learn1 = ConvLearner.pretrained(arch1, data1, precompute=False)
learn1.precompute = False

In [8]:
data2 = ImageClassifierData.from_paths(PATH2, tfms=tfms_from_model(arch2, sz2))
learn2 = ConvLearner.pretrained(arch2, data2, precompute=False)
learn2.precompute = False

In [103]:
learn1.load('1000x1000_256_resnet50_staintools')

In [10]:
learn2.load('resnet50_new_patches_3000_staintools')

In [11]:
learn2.model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1

In [104]:
class ResNet50Bottom(nn.Module):
    def __init__(self, original_model):
        super(ResNet50Bottom, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-8])
        
    def forward(self, x):
        x = self.features(x)
        return x

model2_layer9 = ResNet50Bottom(learn2.model)
for param in model2_layer9.parameters():
    param.requires_grad = False

model1_layer9 = ResNet50Bottom(learn1.model)
for param in model1_layer9.parameters():
    param.requires_grad = False

In [92]:
# MODEL 2 TRAIN
data_model2_trn = pd.DataFrame(columns = list(range(4096)).append('fname'))

n = data2.trn_dl.dataset.get_n()

for index, name in enumerate(data2.trn_dl.dataset.fnames):
    img = data2.trn_dl.get_batch([index])[0]
    output = model2_layer9(Variable(V(img)))
    
    if (index % 100 == 0):
        print (index)
    
    a = output[0].cpu().numpy()
    tmp = pd.DataFrame(data=a).T
    tmp['fname'] = [name]
    data_model2_trn = pd.concat([data_model2_trn, tmp], ignore_index=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800


In [93]:
# MODEL 2 VAL
data_model2_val = pd.DataFrame(columns = list(range(4096)).append('fname'))

fnames = data2.val_dl.dataset.fnames

for index, name in enumerate(fnames):
    img = data2.val_dl.get_batch([index])[0]
    output = model2_layer9(Variable(V(img)))
    
    if (index % 100 == 0):
        print (index)
    
    a = output[0].cpu().numpy()
    tmp = pd.DataFrame(data=a).T
    tmp['fname'] = [name]
    data_model2_val = pd.concat([data_model2_val, tmp], ignore_index=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500


In [105]:
# MODEL 1 TRAIN
data_model1_trn = pd.DataFrame(columns = list(range(4096)).append('fname'))

n = data1.trn_dl.dataset.get_n()

for index, name in enumerate(data1.trn_dl.dataset.fnames):
    img = data1.trn_dl.get_batch([index])[0]
    output = model1_layer9(Variable(V(img)))
    
    if (index % 100 == 0):
        print (index)
    
    a = output[0].cpu().numpy()
    tmp = pd.DataFrame(data=a).T
    tmp['fname'] = [name]
    data_model1_trn = pd.concat([data_model1_trn, tmp], ignore_index=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [112]:
# MODEL 1 VAL
data_model1_val = pd.DataFrame(columns = list(range(4096)).append('fname'))

fnames = data1.val_dl.dataset.fnames

for index, name in enumerate(fnames):
    img = data1.val_dl.get_batch([index])[0]
    output = model1_layer9(Variable(V(img)))
    
    if (index % 100 == 0):
        print (index)
    
    a = output[0].cpu().numpy()
    tmp = pd.DataFrame(data=a).T
    tmp['fname'] = [name]
    data_model1_val = pd.concat([data_model1_val, tmp], ignore_index=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [107]:
data_model1_trn['target'] = data1.trn_y

In [113]:
data_model1_val['target'] = data1.val_y

In [99]:
data_model2_trn['target'] = data2.trn_y

In [100]:
data_model2_val['target'] = data2.val_y

In [ ]:
data_model2_trn.to_csv('../results/data_3000x3000_train_resnet50_layer9.csv')

In [ ]:
data_model2_val.to_csv('../results/data_3000x3000_valid_resnet50_layer9.csv')

In [109]:
data_model1_trn.to_csv('../results/data_1000x1000_train_resnet50_layer9.csv')

In [114]:
data_model1_val.to_csv('../results/data_1000x1000_valid_resnet50_layer9.csv')

In [111]:
len(data1.val_dl.dataset.fnames)

30165

In [152]:
i = iter(data2.trn_dl)
inputs, labels = next(i)
inputs, labels = Variable(inputs), Variable(labels)
outputs = res50_conv2(inputs)

In [110]:
len(data2.trn_dl.dataset.fnames)

9819

In [120]:
outputs.shape

torch.Size([64, 4096])

In [ ]:
outputs[1]

In [122]:
inputs, labels = next(i)
inputs, labels = Variable(inputs), Variable(labels)
outputs = res50_conv2(inputs)
outputs[0]

tensor([0.8885, 3.4051, 1.9177,  ..., 0.5425, 0.2665, 0.6057], device='cuda:0')

In [ ]:
i = iter(data2.trn_dl)

In [79]:
a = model2_layer9(Variable(V(data2.trn_dl.get_batch([9818, 1])[0])))

In [36]:
a = a[0].cpu().numpy()

In [57]:
pd.DataFrame(data=a).T

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,2.081404,6.544422,2.773311,3.486383,4.855809,4.829822,4.377133,4.187634,2.618723,2.574072,...,0.386802,0.423455,0.41649,0.38185,0.404462,0.192222,0.421217,0.333867,0.452976,0.318715


In [86]:
data2.trn_dl.get_batch(list(range(0, 63)))[0]

array([[[[ 2.20931,  2.2084 ,  2.2315 , ...,  2.21325,  2.21308,  2.19803],
         [ 2.20635,  2.22136,  2.213  , ...,  2.20036,  2.21115,  2.20583],
         [ 2.21964,  2.2162 ,  2.21311, ...,  2.22099,  2.21844,  2.2132 ],
         ...,
         [ 2.16112,  2.14096,  1.3836 , ...,  2.18119,  2.20821,  2.2176 ],
         [ 2.15107,  2.13198,  2.00224, ...,  2.18312,  2.18173,  2.19945],
         [ 1.76234,  2.13101,  2.18604, ...,  2.19154,  2.19191,  2.19156]],

        [[ 2.38809,  2.38716,  2.41078, ...,  2.39212,  2.39194,  2.37656],
         [ 2.38507,  2.40041,  2.39186, ...,  2.37894,  2.38997,  2.38453],
         [ 2.39083,  2.38607,  2.38573, ...,  2.40003,  2.39402,  2.38479],
         ...,
         [ 2.2071 ,  2.21112,  0.86038, ...,  2.2718 ,  2.30169,  2.31049],
         [ 2.22089,  2.131  ,  1.88265, ...,  2.27378,  2.2808 ,  2.29047],
         [ 1.57282,  2.17888,  2.21414, ...,  2.28238,  2.3007 ,  2.28241]],

        [[ 2.5997 ,  2.59878,  2.62229, ...,  2.60371,  

In [176]:
np.array(data2.trn_dl.dataset.get1item(12)[0])

array([[[ 1.65136,  1.59942,  1.43624, ...,  0.93642,  1.17164,  0.75017],
        [ 1.38131,  1.41287,  1.16173, ...,  0.94259,  0.8877 ,  0.11419],
        [ 1.2332 ,  1.11449,  1.39986, ...,  0.73916,  0.68656,  0.19979],
        ...,
        [ 2.23416,  2.23951,  2.22591, ...,  2.23541,  2.23908,  2.22926],
        [ 2.22168,  2.23207,  2.22627, ...,  2.23016,  2.23426,  2.22651],
        [ 2.22473,  2.22475,  2.23496, ...,  2.23063,  2.22432,  2.23869]],

       [[ 0.9711 ,  0.93752,  0.5783 , ..., -0.31907,  0.08526, -0.34831],
        [ 0.45204,  0.40131,  0.08633, ..., -0.32827, -0.38344, -1.0391 ],
        [ 0.16897,  0.02024,  0.54729, ..., -0.57599, -0.60046, -1.01487],
        ...,
        [ 2.41349,  2.41896,  2.40506, ...,  2.41477,  2.41852,  2.40848],
        [ 2.40074,  2.41136,  2.40279, ...,  2.4094 ,  2.41046,  2.39499],
        [ 2.40386,  2.40387,  2.40569, ...,  2.40989,  2.39188,  2.38945]],

       [[ 2.14987,  2.10124,  1.96512, ...,  1.58498,  1.80771,  1.100

In [174]:
data2.trn_dl.get_batch([12])[0]

array([[[[ 1.65136,  1.59942,  1.43624, ...,  0.93642,  1.17164,  0.75017],
         [ 1.38131,  1.41287,  1.16173, ...,  0.94259,  0.8877 ,  0.11419],
         [ 1.2332 ,  1.11449,  1.39986, ...,  0.73916,  0.68656,  0.19979],
         ...,
         [ 2.23416,  2.23951,  2.22591, ...,  2.23541,  2.23908,  2.22926],
         [ 2.22168,  2.23207,  2.22627, ...,  2.23016,  2.23426,  2.22651],
         [ 2.22473,  2.22475,  2.23496, ...,  2.23063,  2.22432,  2.23869]],

        [[ 0.9711 ,  0.93752,  0.5783 , ..., -0.31907,  0.08526, -0.34831],
         [ 0.45204,  0.40131,  0.08633, ..., -0.32827, -0.38344, -1.0391 ],
         [ 0.16897,  0.02024,  0.54729, ..., -0.57599, -0.60046, -1.01487],
         ...,
         [ 2.41349,  2.41896,  2.40506, ...,  2.41477,  2.41852,  2.40848],
         [ 2.40074,  2.41136,  2.40279, ...,  2.4094 ,  2.41046,  2.39499],
         [ 2.40386,  2.40387,  2.40569, ...,  2.40989,  2.39188,  2.38945]],

        [[ 2.14987,  2.10124,  1.96512, ...,  1.58498,  